## Getting started
Set working directory to top level of repo to ensure links work correctly:

In [1]:
cd ..

/home/jovyan/Robbi/dea-intertidal


Install additional packages directly from the requirements file

In [ ]:
# pip install -r requirements.in --quiet

### Load packages

In [8]:
%load_ext autoreload
%autoreload 2

from IPython.display import Image

# import datacube
import glob
import xarray as xr
import pandas as pd
import numpy as np

# import datacube
# from datacube.utils.geometry import assign_crs
# import odc.geo.xr
from dea_tools.plotting import xr_animation
# from intertidal.utils import load_config, intertidal_hillshade

# Connect to datacube
# dc = datacube.Datacube(app="Intertidal_elevation")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
xr.open_rasterio('data/interim/burdekin_2016_2018_elevation.tif').geobox

/tmp/ipykernel_488/2269062481.py:1: DeprecationWarning: open_rasterio is Deprecated in favor of rioxarray. For information about transitioning, see: https://corteva.github.io/rioxarray/stable/getting_started/getting_started.html
  xr.open_rasterio('data/interim/burdekin_2016_2018_elevation.tif').geobox
/env/lib/python3.8/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


GeoBox(1231, 1370, Affine(10.0, 0.0, 1616560.0,
       0.0, -10.0, -2199900.0), +init=epsg:3577 +type=crs)

In [3]:
name = "burdekin"
elevations = sorted(glob.glob(f"data/interim/{name}*elevation.tif"))
extents = sorted(glob.glob(f"data/interim/{name}*extents.tif"))

vmin, vmax = xr.open_rasterio(elevations[0]).quantile([0.02, 0.95]).values

out_list = []
for elevation, extent in zip(elevations, extents):
    out_list.append(
        intertidal_hillshade(
            elevation,
            extent,
            altdeg=50,
            vert_exag=300,
            vmin=vmin,
            vmax=vmax,
            fraction=1.0,
        )
    )

intertidal_da = xr.concat(
    out_list, dim=pd.DatetimeIndex([str(int(i.split("_")[-2]) + 1) for i in elevations])
).rename({"concat_dim": "time"})
intertidal_da = intertidal_da.odc.assign_crs("EPSG:3577")

/tmp/ipykernel_488/2187324549.py:5: DeprecationWarning: open_rasterio is Deprecated in favor of rioxarray. For information about transitioning, see: https://corteva.github.io/rioxarray/stable/getting_started/getting_started.html
  vmin, vmax = xr.open_rasterio(elevations[0]).quantile([0.02, 0.95]).values


NameError: name 'intertidal_hillshade' is not defined

In [ ]:
intertidal_da.geobox

In [ ]:
hltc = dc.load(product='low_tide_comp_20p', 
        like=intertidal_da.odc.geobox.compat, 
        resampling='bilinear',
        measurements=['red', 'green', 'blue', 'nir']) * 2
hltc['nir'][:] = 1
hltc = hltc.rename({'nir': 'alpha'})

intertidal_ds = xr.where(intertidal_da.isnull(), hltc.squeeze('time').to_array('variables'), intertidal_da).to_dataset('variables')
intertidal_ds = intertidal_ds.odc.assign_crs("EPSG:3577")
intertidal_ds

In [ ]:
intertidal_ds.geobox

In [ ]:
testing = intertidal_ds.interp({'time': pd.date_range(intertidal_ds.time.min().values, 
                                                      intertidal_ds.time.max().values, freq='1M')})

In [ ]:
xr_animation(
    ds=testing.isel(x=slice(500, 1100), y=slice(0, 600)),
    bands=["red", "green", "blue"],
    imshow_kwargs={"vmin": 0.0, "vmax": 1.0, "interpolation": "bilinear"},
    output_path="animation.gif",
    interval=20,
    show_date='%Y'
)
Image("animation.gif")